# Installs & imports

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 46.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install fasttext-wheel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 30.4 MB/s eta 0:00:00


In [1]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.5 MB/s eta 0:00:00


In [2]:
!pip install datasets evaluate accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 26.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl

In [4]:
import os
from typing import List, Tuple, Optional

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

import spacy
import gensim.downloader
import fasttext

import wandb

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

from datasets import Dataset
from transformers import DataCollatorWithPadding
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import AutoModel, BertTokenizerFast, BertModel, BertForSequenceClassification, pipeline

# Data

In [ ]:
!gdown https://drive.google.com/drive/folders/1Df8XPJNz2k2O8K_7NJ9hPBceRB6oa8gB --folder

Retrieving folder contents
Processing file 1XJ7qL-6B6Cu7seQ54g-eufW4HPrwMld4 test_spam.csv
Processing file 1ZAfTp1mvGlIP3wdkKHWuq_OqGdXSi2_F train_spam.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1XJ7qL-6B6Cu7seQ54g-eufW4HPrwMld4
To: /content/spam_dataset/test_spam.csv
100% 1.34M/1.34M [00:00<00:00, 158MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZAfTp1mvGlIP3wdkKHWuq_OqGdXSi2_F
To: /content/spam_dataset/train_spam.csv
100% 5.40M/5.40M [00:00<00:00, 136MB/s]
Download completed


In [ ]:
train_data = pd.read_csv('spam_dataset/train_spam.csv', header=0, index_col=False)
test_data = pd.read_csv('spam_dataset/test_spam.csv', header=0, index_col=False)

In [ ]:
train_data.shape, test_data.shape

((16278, 2), (4070, 1))

In [ ]:
train_data.head()

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...


In [ ]:
test_data.head()

,text
0,j jim whitehead ejw cse ucsc edu writes j you ...
1,original message from bitbitch magnesium net p...
2,java for managers vince durasoft who just taug...
3,there is a youtuber name saiman says
4,underpriced issue with high return on equity t...


In [ ]:
train_data['text_type'].value_counts()

text_type
ham     11469
spam     4809
Name: count, dtype: int64

In [ ]:
train_data['text_type'] = train_data['text_type'].apply(lambda x: 1 if x == 'spam' else 0)

In [ ]:
train_ids, val_ids = train_test_split(train_data.index, test_size=0.2, stratify=train_data.text_type, random_state=74)

In [ ]:
train_set, val_set = train_data.loc[train_ids], train_data.loc[val_ids]
train_set.text_type.value_counts() / train_set.shape[0], val_set.text_type.value_counts() / val_set.shape[0]

(text_type
 0    0.704577
 1    0.295423
 Name: count, dtype: float64,
 text_type
 0    0.704545
 1    0.295455
 Name: count, dtype: float64)

# Preprocessing

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def preprocessing(text):
    parsed = nlp(text)
    normalized = []
    for token in parsed:
        if token.is_stop or token.is_punct or token.is_space:
            continue
        normalized.append(token.lemma_)
    return ' '.join(normalized)

In [ ]:
%%time
train_set['preprocessed'] = train_set.text.apply(preprocessing)
val_set['preprocessed'] = val_set.text.apply(preprocessing)
test_data['preprocessed'] = test_data.text.apply(preprocessing)

CPU times: user 4min 17s, sys: 736 ms, total: 4min 18s
Wall time: 4min 33s


In [ ]:
train_set.to_csv('preprocessed_train.csv', header=True, index=True)
val_set.to_csv('preprocessed_val.csv', header=True, index=True)
test_data.to_csv('preprocessed_test.csv', header=True, index=True)

In [5]:
train_set = pd.read_csv('preprocessed_train.csv', header=0, index_col=0).fillna('')
val_set = pd.read_csv('preprocessed_val.csv', header=0, index_col=0).fillna('')
test_data = pd.read_csv('preprocessed_test.csv', header=0, index_col=0).fillna('')


# 1: Vectorization + classification

## Vectorizers

In [ ]:
embeddings = {}

### TF-IDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = tfidf_vectorizer.fit_transform(train_set.preprocessed)
X_val_tfidf = tfidf_vectorizer.transform(val_set.preprocessed)

In [ ]:
X_train_tfidf.shape, X_val_tfidf.shape

((13022, 10000), (3256, 10000))

In [ ]:
embeddings['tfidf'] = {'train': X_train_tfidf, 'val': X_val_tfidf}

### Word2Vec

In [ ]:
w2v_vectors = gensim.downloader.load('word2vec-google-news-300')

In [ ]:
class W2vVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, w2v_model):
        self._model = w2v_model

    def fit(self, X, y=None):
        self.is_fitted_ = True
        return self

    def transform(self, X) -> np.ndarray:
        vectors = []
        for sample in X:
            if sample:
                vector = self._model.get_mean_vector(
                    sample.split(),
                    pre_normalize=False,
                    ignore_missing=True
                    )
            else:
                vector = np.zeros(self._model.vector_size)
            vectors.append(vector)
        return np.array(vectors)

In [ ]:
w2v_vectorizer = W2vVectorizer(w2v_vectors)

In [ ]:
X_train_w2v = w2v_vectorizer.fit_transform(train_set.preprocessed)
X_val_w2v = w2v_vectorizer.transform(val_set.preprocessed)
X_train_w2v.shape, X_val_w2v.shape

((13022, 300), (3256, 300))

In [ ]:
embeddings['w2v'] = {'train': X_train_w2v, 'val': X_val_w2v}

In [ ]:
del w2v_vectors
del w2v_vectorizer

### FastText

In [ ]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
# !gunzip cc.en.300.bin.gz
ft_model = fasttext.load_model('cc.en.300.bin')

In [ ]:
class FastTextVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, ft_model):
        self._model = ft_model

    def fit(self, X, y=None):
        self.is_fitted_ = True
        return self

    def transform(self, X) -> np.ndarray:
        vectors = []
        for i, sample in enumerate(X):
            vector = self._model.get_sentence_vector(sample)
            vectors.append(vector)
        return np.array(vectors)

In [ ]:
ft_vectorizer = FastTextVectorizer(ft_model)

In [ ]:
X_train_ft = ft_vectorizer.fit_transform(train_set.preprocessed)
X_val_ft = ft_vectorizer.transform(val_set.preprocessed)
X_train_ft.shape, X_val_ft.shape

((13022, 300), (3256, 300))

In [ ]:
embeddings['ft'] = {'train': X_train_ft, 'val': X_val_ft}

In [ ]:
del ft_model
del ft_vectorizer

---

In [ ]:
import pickle

with open('embeddings.pkl', 'wb') as newf:
    pickle.dump(embeddings, newf)

# with open('embeddings.pkl', 'rb') as f:
#     embeddings = pickle.load(f)

## Classifiers

In [ ]:
classifiers  = {
    'logreg': LogisticRegression(random_state=74),
    'nb': MultinomialNB(),
    'rf@10': RandomForestClassifier(max_depth=10, random_state=74),
    'rf@100': RandomForestClassifier(max_depth=100, random_state=74),
    'rf@1000': RandomForestClassifier(max_depth=1000, random_state=74)
}

## Cross-validation

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
sweep_configuration = {
    "method": "grid",
    "metric": {"goal": "maximize", "name": "val_auc_roc"},
    "parameters": {
        "embeddings": {"values": ['tfidf', 'w2v', 'ft']},
        "classifiers": {"values": ['logreg', 'nb', 'rf@10', 'rf@100', 'rf@1000']},
    },
}

In [ ]:
def train():
    run = wandb.init()

    if wandb.config.classifiers == 'nb' and wandb.config.embeddings in ['ft', 'w2v']:
        return

    embs = embeddings[wandb.config.embeddings]
    clf = classifiers[wandb.config.classifiers]

    clf.fit(embs['train'], train_set.text_type.to_numpy())

    train_pred = clf.predict_proba(embs['train'])
    train_auc_roc = roc_auc_score(train_set.text_type.to_numpy(), train_pred[:, 1])

    val_pred = clf.predict_proba(embs['val'])
    val_auc_roc = roc_auc_score(val_set.text_type.to_numpy(), val_pred[:, 1])
    wandb.log({
        'train_auc_roc': train_auc_roc,
        'val_auc_roc': val_auc_roc})

In [ ]:
pipe1_sweep = wandb.sweep(sweep=sweep_configuration, project="vk_spam_detection")
wandb.agent(pipe1_sweep, function=train)

Create sweep with ID: vcw608nb
Sweep URL: https://wandb.ai/vknyazkova/vk_spam_detection/sweeps/vcw608nb


wandb: Agent Starting Run: kei2fi0w with config:
wandb: 	classifiers: logreg
wandb: 	embeddings: tfidf


train_auc_roc,▁
val_auc_roc,▁
train_auc_roc,0.98902
val_auc_roc,0.98066


wandb: Agent Starting Run: 5tec1swy with config:
wandb: 	classifiers: logreg
wandb: 	embeddings: w2v


train_auc_roc,▁
val_auc_roc,▁
train_auc_roc,0.93448
val_auc_roc,0.93014


...

wandb: Agent Starting Run: 4fdw9pz9 with config:
wandb: 	classifiers: rf@1000
wandb: 	embeddings: w2v


train_auc_roc,▁
val_auc_roc,▁
train_auc_roc,0.99946
val_auc_roc,0.95007


wandb: Agent Starting Run: iwcfrmvt with config:
wandb: 	classifiers: rf@1000
wandb: 	embeddings: ft


train_auc_roc,▁
val_auc_roc,▁
train_auc_roc,0.99987
val_auc_roc,0.97243


wandb: Sweep Agent: Waiting for job.
wandb: Ctrl + C detected. Stopping sweep.


# 2: Using spam fine-tuned BERT

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [46]:
bert_classifier = pipeline("text-classification",
                           model="DunnBC22/distilbert-base-uncased-SpamFilter-LG",
                           padding='longest',
                           device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [47]:
train_dataset = Dataset.from_pandas(train_set.rename(columns={'text_type': 'labels'}), split='train')
val_dataset = Dataset.from_pandas(val_set.rename(columns={'text_type': 'labels'}), split='test')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
spam_probs = []
true_labels = []
for samples in val_loader:
    preds = bert_classifier(samples['text'], return_all_scores=True)
    for pred in preds:
        spam_probs.append(pred[1]['score'])
    true_labels.extend(samples['labels'].tolist())

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
roc_auc_score(true_labels, spam_probs)

0.7930908072582004

# 3: Fine-tune BERT on train data

In [78]:
data_collator = DataCollatorWithPadding(tokenizer=bert_classifier.tokenizer, return_tensors="pt", padding='longest')
rocauc = evaluate.load("roc_auc")

In [49]:
def preprocess_function(examples):
    return bert_classifier.tokenizer(examples["text"], truncation=True, padding=True, return_tensors='pt')

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/13022 [00:00<?, ? examples/s]

Map:   0%|          | 0/3256 [00:00<?, ? examples/s]

In [50]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    probs = F.softmax(torch.tensor(predictions), dim=1)
    return rocauc.compute(references=labels, prediction_scores=probs[:, 1])

In [51]:
os.environ["WANDB_PROJECT"] = "vk_spam_detection"

In [54]:
training_args = TrainingArguments(
    output_dir='bert_spam_detection',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    report_to="wandb",
    logging_steps=1,
)

trainer = Trainer(
    model=bert_classifier.model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=bert_classifier.tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: Currently logged in as: vknyazkova. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Roc Auc
1,0.011300,0.097391,0.994784
2,0.000900,0.107873,0.995110


TrainOutput(global_step=1628, training_loss=0.09690082233822624, metrics={'train_runtime': 1192.6511, 'train_samples_per_second': 21.837, 'train_steps_per_second': 1.365, 'total_flos': 3208149275531712.0, 'train_loss': 0.09690082233822624, 'epoch': 2.0})

In [56]:
!pip install huggingface_hub

In [57]:
from huggingface_hub import notebook_login

notebook_login()

In [90]:
trainer.push_to_hub('vk_spam_detection_bert')

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vknyazkova01/bert_spam_detection/commit/2c67cfd51189733b2b2c2f7ee39b42b4a693ab37', commit_message='vk_spam_detection_bert', commit_description='', oid='2c67cfd51189733b2b2c2f7ee39b42b4a693ab37', pr_url=None, pr_revision=None, pr_num=None)

# Inference
Поскольку лучшее качество на валидации дал дообученный берт, на тесте предсказания буду делать им

In [91]:
my_bert_classifier = pipeline("text-classification",
                              model='vknyazkova01/bert_spam_detection',
                              device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [92]:
test_dataset = Dataset.from_pandas(test_data, split='test')
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [103]:
# на всякий случай перепроверю такой ли скор на валидации
spam_probs = []
true_labels = []
with torch.no_grad():
    for samples in val_loader:
        preds = my_bert_classifier(samples['text'], return_all_scores=True)
        for pred in preds:
            spam_probs.append(pred[1]['score'])
        true_labels.extend(samples['labels'].tolist())
rocauc.compute(references=true_labels, prediction_scores=spam_probs)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


{'roc_auc': 0.9951101762348493}

In [108]:
test_scores = []
with torch.no_grad():
    for samples in test_loader:
        preds = my_bert_classifier(samples['text'], return_all_scores=True)
        for pred, text in zip(preds, samples['text']):
            test_scores.append((pred[1]['score'], text))
test_scores = pd.DataFrame(test_scores, columns=['score', 'text'])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [110]:
test_scores.to_csv('predicted_test.csv', header=True, index=False)